# Imports

In [4]:
import requests
import bs4
import json
import datetime as dt
import sys
import pandas as pd
import geopy
import warnings
warnings.filterwarnings('ignore')

# Lisbon Properties for Sale - SUPERCASA Webscrapping

In [5]:
url = 'https://supercasa.pt/comprar-casas/lisboa/pagina-1'
result = requests.get(url)
soup = bs4.BeautifulSoup(result.text, 'lxml')
num_of_properties = int(soup.find_all('h1', id='searchTitle')[0].get_text().split()[0].replace('.',''))
num_prop_per_page = 25
total_pages = int(num_of_properties / num_prop_per_page)
time_estimation_for_12505 = 20
time_estimation = round(num_of_properties * 20 / 12505)

print(f'Time estimation for this task: {time_estimation} minutes.')
estimation = (dt.datetime.now() + dt.timedelta(minutes=time_estimation))
start = f'Tasks started at {dt.datetime.now().hour}h:{dt.datetime.now().minute}min. Estimated finish time {estimation.hour}h:{estimation.minute}min.'
print(start)

title, price, num_rooms, total_area, latitude, longitude, region, extras, id = ([] for i in range(9))

for n in range(1, 75):
    url = f'https://supercasa.pt/comprar-casas/lisboa/pagina-{n}'
    result = requests.get(url)
    soup = bs4.BeautifulSoup(result.text, 'lxml')

    # Find all properties on the current page
    properties = soup.find_all('div', class_='property big-picture') 

    for prop in properties:
        # Title
        a = prop.find('h2', class_='property-list-title').find('a')
        title.append(a.get_text().strip() if a else ' ')

        # Price
        span = prop.find('div', class_='property-price').find('span')
        price.append(span.get_text(strip=True) if span else ' ')

        # Features
        feature = prop.find('div', class_='property-features')
        spans = feature.find_all('span') if feature else []
        rooms = spans[0].get_text() if len(spans) > 0 else "Unknown"
        area = spans[1].get_text() if len(spans) > 1 else "Unknown"
        num_rooms.append(rooms)
        total_area.append(area)

        # Links for latitude and longitude
        link = prop.find('a', class_='property-link')
        latitude.append(link.get('data-latitude') if link else 'Unknown')
        longitude.append(link.get('data-longitude') if link else 'Unknown')

        # Extras
        highlight = prop.find('div', class_='property-highlights')
        if highlight:
            extra_spans = highlight.find_all('span')
            extras.append(', '.join([span.get_text(strip=True) for span in extra_spans]))
        else:
            extras.append(' ')

        # Address region from JSON-LD script if necessary
        script = prop.find('script', type='application/ld+json')
        if script:
            data = json.loads(script.string)
            if data.get('@type') == 'Offer':
                available_at_or_from = data.get('availableAtOrFrom', {})
                address_info = available_at_or_from.get('address', {})
                address_region = address_info.get('addressRegion', 'Not provided')
                region.append(address_region)
        else:
            region.append('Not provided')

        sys.stdout.write(f"\rProgress: {int((n / total_pages) * 100)}%")
        sys.stdout.flush()

for i in range(len(title)):
    id.append(i)

sys.stdout.write(f"\rProgress: 100%")
sys.stdout.flush()
print('\nCompleted!')

headers = ['id', 'title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'region', 'extras']
final_data = [id, title, price, num_rooms, total_area, latitude, longitude, region, extras]
Lisbon_Properties = pd.DataFrame(dict(zip(headers, final_data)))

print(f"\nYou now have data on {len(Lisbon_Properties['id'])} properties located in Lisbon!")

Time estimation for this task: 20 minutes.
Tasks started at 19h:31min. Estimated finish time 19h:51min.
Progress: 100%
Completed!

You now have data on 1376 properties located in Lisbon!


In [6]:
# Lisbon_Properties.to_csv('Lisbon_Properties.csv')

In [7]:
# Lisbon_Properties = pd.read_csv('Lisbon_Properties.csv', index_col=0)
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
1,1,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
2,2,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
3,3,"Apartamento T2+1 em Calçada do Combro, Miseric...",1.800.000 €,2 quartos,Área bruta 234 m²,"38,71101","-9,14865",Misericórdia,"Com elevador, Com garagem"
4,4,"Apartamento T5 na Rua Sousa Pinto, Santo Antón...",3.950.000 €,5 quartos,Área bruta 416 m²,"38,7234746455","-9,1581178942",Santo António,"De luxo, Com garagem"


In [8]:
Lisbon_Properties = Lisbon_Properties[~Lisbon_Properties.drop('id', axis=1).duplicated()]

In [9]:
Lisbon_Properties['num_rooms'].value_counts()

num_rooms
2 quartos                   430
1 quarto                    314
3 quartos                   264
4 quartos                   138
5 quartos                    29
9 quartos                    12
Área bruta 63 m²              9
6 quartos                     6
7 quartos                     6
Área bruta 41 m²              4
Área bruta 46 m²              4
Área bruta 60 m²              3
Área bruta 51 m²              3
Área bruta 30 m²              3
Área bruta 33 m²              3
Área útil 63 m²               3
Área bruta 25 m²              2
C.E.: D                       2
Área bruta 35 m²              2
10 quartos                    2
Área bruta 50 m²              2
Área bruta 37 m²              2
Área bruta 28 m²              2
Área bruta 68 m²              2
Área bruta 52 m²              1
Área útil 118 m²              1
Área bruta 31 m²              1
Área útil 41 m²               1
Área bruta 18 m²              1
Área bruta 39 m²              1
Área bruta 32 m²              

In [10]:
Lisbon_Properties['total_area'].unique()

array(['Área bruta 90 m²', 'Área bruta 120 m²', 'Área bruta 97 m²',
       'Área bruta 234 m²', 'Área bruta 416 m²', 'Área útil 233 m²',
       'Área bruta 93 m²', 'Área bruta 55 m²', 'Área bruta 290 m²',
       'Área bruta 336 m²', 'Área bruta 262 m²', 'Área bruta 874 m²',
       'Área bruta 264 m²', 'Área bruta 60 m²', 'Área útil 88 m²',
       'Área bruta 68 m²', 'Área bruta 48 m²', 'Área bruta 118 m²',
       'Área bruta 175 m²', 'Área bruta 105 m²', 'Área útil 140 m²',
       'Área bruta 133 m²', 'Área bruta 89 m²', 'Área bruta 168 m²',
       'Área bruta 85 m²', 'Área bruta 44 m²', 'Área bruta 67 m²',
       'Área útil 139 m²', 'C.E.: D', 'Área bruta 150 m²',
       'Área bruta 160 m²', 'Área bruta 426 m²', 'Área bruta 70 m²',
       'Área bruta 102 m²', 'Área bruta 54 m²', 'Área bruta 110 m²',
       'C.E.: A', 'Área útil 142 m²', 'Área bruta 198 m²',
       'Área bruta 302 m²', 'Área bruta 138 m²', 'Área bruta 130 m²',
       'Unknown', 'Área bruta 134 m²', 'Área bruta 49 m²',


# Lisbon Metro Info - Wikipedia Webscrapping

In [25]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_esta%C3%A7%C3%B5es_do_Metropolitano_de_Lisboa'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
data = str(soup.find('table', {'class': 'wikitable'}))

table = pd.read_html(data)[0]
columns = ['Nome','Outros nomes','Linha','Lat.','Long.']
Lisbon_Metro = table[columns]

Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.
0,Aeroporto,—,Vermelha,38.76861,−9.12861
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261
3,Alfornelos,—,Azul,38.76038,−9.20435
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003


In [27]:
Lisbon_Metro['NomeConcat'] = Lisbon_Metro['Nome']+' - '+Lisbon_Metro['Outros nomes']
Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.,NomeConcat
0,Aeroporto,—,Vermelha,38.76861,−9.12861,Aeroporto - —
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388,Alameda - Alameda I (técn.)
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261,Alameda - Alameda II (técn.)
3,Alfornelos,—,Azul,38.76038,−9.20435,Alfornelos - —
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003,Alto dos Moinhos - Centro Administrativo (prev.)


# Data

In [14]:
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
1,1,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
2,2,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
3,3,"Apartamento T2+1 em Calçada do Combro, Miseric...",1.800.000 €,2 quartos,Área bruta 234 m²,"38,71101","-9,14865",Misericórdia,"Com elevador, Com garagem"
4,4,"Apartamento T5 na Rua Sousa Pinto, Santo Antón...",3.950.000 €,5 quartos,Área bruta 416 m²,"38,7234746455","-9,1581178942",Santo António,"De luxo, Com garagem"


In [15]:
def extra_rooms(x):
    if '+' in x:
        return int(x.split('+')[1])
    else:
        return 0

def area(x):
    if 'Área' in x:
        return int(x.split()[2].replace('.',''))
    else:
        return 'Unknown'

def num_extras(x):
    if x.isspace():
        return 0
    elif ',' in x:
        return int(len(x.split(',')))
    else:
        return 1

In [16]:
Lisbon_Properties['Type'] = Lisbon_Properties['title'].apply(lambda x: x.split()[0])
Lisbon_Properties['Typology'] = Lisbon_Properties['title'].apply(lambda x: x.split()[1])
Lisbon_Properties['Extra_Rooms'] = Lisbon_Properties['Typology'].apply(extra_rooms)
Lisbon_Properties['Extra_Rooms_Flag'] = Lisbon_Properties['Extra_Rooms'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['N_Rooms'] = Lisbon_Properties['num_rooms'].apply(lambda x: int(x.split()[0]))
Lisbon_Properties['Total_N_Rooms'] = Lisbon_Properties['N_Rooms'] + Lisbon_Properties['Extra_Rooms']
Lisbon_Properties['Price'] = Lisbon_Properties['price'].apply(lambda x: int(x.replace('.','').split()[0]))
Lisbon_Properties['Area_m2'] = Lisbon_Properties['total_area'].apply(area)
Lisbon_Properties['N_Extras'] = Lisbon_Properties['extras'].apply(num_extras)
Lisbon_Properties['Extras_Flag'] = Lisbon_Properties['N_Extras'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['Latitude'] = Lisbon_Properties['latitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties['Longitude'] = Lisbon_Properties['longitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties.rename(columns={'region':'Region'}, inplace=True)

ValueError: invalid literal for int() with base 10: 'Área'

In [17]:
extras_df = Lisbon_Properties['extras'].str.split(',', expand=True)

extras = []
unique_extras = []

for x in [0,1,2,3,4]:
    extras.append(list(extras_df[x].unique()))

for ext in extras:
    unique_extras += ext

unique_extras = set(unique_extras)
unique_extras = {extra.strip() for extra in unique_extras if extra is not None and extra.strip()}

unique_extras


{'Com elevador',
 'Com garagem',
 'De luxo',
 'Piscina',
 'Rés do chão',
 'Vista para mar',
 'Último andar'}

In [18]:
for feature in unique_extras:
    Lisbon_Properties[feature] = Lisbon_Properties['extras'].apply(lambda x: int(feature in x))

In [19]:
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras,Type,Typology,Extra_Rooms,Extra_Rooms_Flag,Rés do chão,Piscina,De luxo,Com elevador,Vista para mar,Com garagem,Último andar
0,0,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,,Apartamento,T2,0,0,0,0,0,0,0,0,0
1,1,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,,Apartamento,T3,0,0,0,0,0,0,0,0,0
2,2,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,,Apartamento,T1,0,0,0,0,0,0,0,0,0
3,3,"Apartamento T2+1 em Calçada do Combro, Miseric...",1.800.000 €,2 quartos,Área bruta 234 m²,"38,71101","-9,14865",Misericórdia,"Com elevador, Com garagem",Apartamento,T2+1,1,1,0,0,0,1,0,1,0
4,4,"Apartamento T5 na Rua Sousa Pinto, Santo Antón...",3.950.000 €,5 quartos,Área bruta 416 m²,"38,7234746455","-9,1581178942",Santo António,"De luxo, Com garagem",Apartamento,T5,0,0,0,0,1,0,0,1,0


In [20]:
region_dummies = pd.get_dummies(Lisbon_Properties['Region'],dtype=int)
type_dummies = pd.get_dummies(Lisbon_Properties['Type'],dtype=int)

KeyError: 'Region'

In [ ]:
Lisbon_Properties = pd.concat([Lisbon_Properties, region_dummies, type_dummies], axis=1)

In [ ]:
Lisbon_Properties.drop(['Region','Type'], axis=1, inplace=True)

In [ ]:
Lisbon_Properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1195 entries, 0 to 1380
Data columns (total 51 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1195 non-null   int64  
 1   Extra_Rooms              1195 non-null   int64  
 2   Extra_Rooms_Flag         1195 non-null   int64  
 3   N_Rooms                  1195 non-null   int64  
 4   Total_N_Rooms            1195 non-null   int64  
 5   Price                    1195 non-null   int64  
 6   Area_m2                  1195 non-null   int64  
 7   N_Extras                 1195 non-null   int64  
 8   Extras_Flag              1195 non-null   int64  
 9   Latitude                 1195 non-null   float64
 10  Longitude                1195 non-null   float64
 11  Vista para mar           1195 non-null   int64  
 12  Com elevador             1195 non-null   int64  
 13  De luxo                  1195 non-null   int64  
 14  Piscina                  1195

In [ ]:
Lisbon_Properties.head()

,id,Extra_Rooms,Extra_Rooms_Flag,N_Rooms,Total_N_Rooms,Price,Area_m2,N_Extras,Extras_Flag,Latitude,...,São Vicente,Apartamento,Casa,Duplex,Flat,Loft,Moradia,Palacete,Penthouse,Quinta
0,0,0,0,1,1,269900,97,0,0,38.751710,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,2,2,430000,90,0,0,38.745739,...,0,1,0,0,0,0,0,0,0,0
2,2,0,0,3,3,399900,120,0,0,38.746570,...,0,1,0,0,0,0,0,0,0,0
3,3,1,1,4,5,1650000,233,2,1,38.707940,...,0,1,0,0,0,0,0,0,0,0
4,4,0,0,1,1,690000,93,1,1,38.710780,...,0,1,0,0,0,0,0,0,0,0


##On stations bring the nb of stations near a house instead of flagging which stations are in the vicinities. 
Alternative is to flag whether or not there's a metro nearby.

In [23]:
unique_stations = []
for x in Lisbon_Metro['Nome'].unique():
    if not x.isspace():
        unique_stations.append(x)

def flatten_list(l):
    for el in l:
        if isinstance(el, list):
            yield from flatten_list(el)
        else:
            yield el

flattened = [item.strip() for sublist in unique_stations for item in (flatten_list(sublist) if isinstance(sublist, list) else sublist.split(','))]

# Extract unique values using a set
unique_stations = list(set(flattened))
print(unique_stations)

['Bela Vista', 'Reboleira', 'Parque', 'Areeiro', 'Baixa-Chiado', 'Pontinha', 'Rato', 'Restauradores', 'Campo Pequeno', 'Olaias', 'Praça de Espanha', 'São Sebastião', 'Cidade Universitária', 'Colégio Militar / Luz', 'Quinta das Conchas', 'Amadora Este', 'Chelas', 'Nome', 'Terreiro do Paço', 'Avenida', 'Carnide', 'Alto dos Moinhos', 'Campo Grande', 'Martim Moniz', 'Odivelas', 'Saldanha', 'Moscavide', 'Alvalade', 'Lumiar', 'Alfornelos', 'Alameda', 'Roma', 'Olivais', 'Santa Apolónia', 'Jardim Zoológico', 'Laranjeiras', 'Marquês de Pombal', 'Entre Campos', 'Cabo Ruivo', 'Rossio', 'Arroios', 'Aeroporto', 'Senhor Roubado', 'Intendente', 'Telheiras', 'Picoas', 'Cais do Sodré', 'Ameixoeira', 'Encarnação', 'Anjos', 'Oriente']


In [24]:
for station in unique_stations:
    Lisbon_Properties[station] = 0

Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras,Type,...,Aeroporto,Senhor Roubado,Intendente,Telheiras,Picoas,Cais do Sodré,Ameixoeira,Encarnação,Anjos,Oriente
0,0,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,,Apartamento,...,0,0,0,0,0,0,0,0,0,0
1,1,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,,Apartamento,...,0,0,0,0,0,0,0,0,0,0
2,2,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,,Apartamento,...,0,0,0,0,0,0,0,0,0,0
3,3,"Apartamento T2+1 em Calçada do Combro, Miseric...",1.800.000 €,2 quartos,Área bruta 234 m²,"38,71101","-9,14865",Misericórdia,"Com elevador, Com garagem",Apartamento,...,0,0,0,0,0,0,0,0,0,0
4,4,"Apartamento T5 na Rua Sousa Pinto, Santo Antón...",3.950.000 €,5 quartos,Área bruta 416 m²,"38,7234746455","-9,1581178942",Santo António,"De luxo, Com garagem",Apartamento,...,0,0,0,0,0,0,0,0,0,0


<html>
<p><strong>Great Circle Distance Formula:</strong></p>
<p>The formula to calculate the great circle or 'as the crow flies' distance between two points on the Earth's surface, given their latitude and longitude is:</p>
<p style="font-family: 'Lucida Console', Monaco, monospace;">
  \( d = 2R \times \sin^{-1}\left(\sqrt{\sin^2\left(\frac{\theta_2 - \theta_1}{2}\right) + \cos \theta_1 \times \cos \theta_2 \times \sin^2\left(\frac{\phi_2 - \phi_1}{2}\right)}\right) \)
</p>
<p>where:</p>
<ul>
  <li><strong>\( (\theta_1, \phi_1) \)</strong> and <strong>\( (\theta_2, \phi_2) \)</strong> – Coordinates of each point (latitude and longitude, respectively);</li>
  <li><strong>R</strong> – Radius of the Earth; and</li>
  <li><strong>d</strong> – Great circle distance between the points.</li>
</l>
</html>


In [ ]:
Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.
0,Aeroporto,—,Vermelha,38.76861,−9.12861
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261
3,Alfornelos,—,Azul,38.76038,−9.20435
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003


##Keep this cell LAST 

In [ ]:
columns_to_drop = ['title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'Typology','extras']
Lisbon_Properties = Lisbon_Properties.drop(columns_to_drop, axis=1)